# Behind the pipeline

In [13]:
from transformers import pipeline
raw_inputs= ["I've finally made it!", "Why is this not working!"]
classifier = pipeline("sentiment-analysis")
classifier(raw_inputs)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998001456260681},
 {'label': 'NEGATIVE', 'score': 0.9997509121894836}]

## Preprocessing with a tokenizer

In [14]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [15]:
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 1045, 1005, 2310, 2633, 2081, 2009,  999,  102],
        [ 101, 2339, 2003, 2023, 2025, 2551,  999,  102,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0]])}


## Going through the model

In [16]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### A high-dimensional vector?

In [17]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 9, 768])


### Model heads: Making sense out of numbers

In [18]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [19]:
print(outputs.logits.shape)

torch.Size([2, 2])


## Postprocessing the output

In [20]:
print(outputs.logits)

tensor([[-4.1149,  4.4030],
        [ 4.6786, -3.6188]], grad_fn=<AddmmBackward0>)


In [21]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=1)
print(predictions)

tensor([[1.9983e-04, 9.9980e-01],
        [9.9975e-01, 2.4909e-04]], grad_fn=<SoftmaxBackward0>)


In [22]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

# Models

## Creating a transformer

In [23]:
from transformers import BertConfig, BertModel
config = BertConfig()
model = BertModel(config)

In [24]:
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

### Different loading methods

In [25]:
# random initialization
from transformers import BertConfig, BertModel
config = BertConfig()
model = BertModel(config)

In [26]:
# initialize from pretrained
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Saving methods

In [27]:
model.save_pretrained("pretrained")

In [28]:
!ls pretrained/

config.json  pytorch_model.bin


## Using a Transformer model for inference

In [29]:
sequences = ["Hello!", "Cool!", "Nice!"]

In [32]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [34]:
tokenizer(sequences , padding=True, truncation=True)

{'input_ids': [[101, 8667, 106, 102], [101, 13297, 106, 102], [101, 8835, 106, 102]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]}

In [35]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [36]:
import torch
model_inputs = torch.tensor(encoded_sequences)

# Tokenizers